# Get/Update Download statistics

This Jupyter Notebook contains code and analysis for analyzing and visualizing data from the PyPI package repository. The notebook utilizes the Google BigQuery API to query and retrieve data, and pandas and matplotlib libraries for data manipulation and visualization.

The notebook is divided into several sections, each represented by a separate code cell. The sections include:

1. Data Retrieval: This section retrieves monthly download statistics for a specific package from the PyPI repository using the BigQuery API.

2. Data Processing: This section processes and combines the retrieved data with the existing data from a CSV file. It performs data cleaning, sorting, and removes duplicate entries.

3. Data Visualization: This section visualizes the processed data using various charts and plots, including bar charts and line plots.

4. Data Export: This section exports the processed data to a CSV file for further analysis or sharing.

#### Dependencies
Please note that this notebook requires the installation of the necessary dependencies, such as pandas, matplotlib, and the Google Big Query SDK.

see [requirements-stats.txt](../requirements-stats.txt) for a list of dependencies.


In [ ]:
import csv

from pathlib import Path

from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
## create service account from BQ_CREDS file
credentials = service_account.Credentials.from_service_account_file("..\.secrets_\BQ_CREDS.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [3]:
def get_monthly_stats(year, month):
    QUERY = f"""
        SELECT
          COUNT(*) AS downloads,
          REGEXP_EXTRACT(file.project, r".*?-(.*?)-(?:.*-)?stubs") AS port,
          REGEXP_EXTRACT(file.project, r".*?-.*?-(?:(.*)-)?stubs") AS board,
          REGEXP_EXTRACT(file.version, r"(.*).post") AS version,
          DATE({year},{month},1) as report_date,
          file.project as project,
          file.version AS version_full,
          -- REGEXP_EXTRACT(file.version, r".*.post(.*)") AS post,
        FROM
          `bigquery-public-data.pypi.file_downloads`
        WHERE
          file.PROJECT LIKE 'micropython-%-stubs' -- Only query the previous month OF history
          AND DATE(timestamp) BETWEEN DATE({year},{month},1)
          AND DATE_ADD(DATE({year},{month},1), INTERVAL 1 MONTH)
          AND details.installer.name <> 'bandersnatch'
        GROUP BY
          port,
          board,
          version,
          project,
          version_full
          -- ,post
        ORDER BY
          `downloads` DESC
        """
    query_job = client.query(QUERY)  # API request

    results = query_job.result()  # Waits for query to finish
    return results

In [4]:
import datetime
stats_list = []

year = 2023

# get the current year and month
year = datetime.datetime.now().year
this_month = datetime.datetime.now().month


In [5]:
# read the current data from the csv file
csv_file = Path(".") / "downloads.csv"
with open(csv_file, "r") as f:
    reader = csv.DictReader(f)
    current_stats_list = [row for row in reader]

# sort the current stats list by date and get the latest date
current_stats_list.sort(key=lambda x: x["report_date"])
last_date = current_stats_list[-1]["report_date"]
print(f"Last date in current data: {last_date}")

# get the month and year of the last date
last_month = int(last_date.split("-")[1])
last_year = int(last_date.split("-")[0])

print (f"Last month: {last_month}, last year: {last_year}")

Last date in current data: 2023-10-01
Last month: 10, last year: 2023


In [6]:


for month in range(last_month, this_month+1):
    print(f"Processing {year}-{month} ... ", end="", flush=True)
    results = get_monthly_stats(year, month)
    field_names = [f.name for f in results.schema]
    # print(field_names)
    print(f"Retrieved {results.total_rows} download summaries")
    for row in results:
        stats_list.append(dict(row))


Processing 2023-10 ... Retrieved 107 download summaries
Processing 2023-11 ... Retrieved 49 download summaries


In [7]:
full_list = current_stats_list + stats_list

In [8]:
# avoid double counts 
unique = {}
for rec in full_list:
    key = f"{rec['report_date']}-{rec['project']}-{rec['version_full']}"
    unique[key] = rec

full_list = list(unique.values())

In [9]:
print("writing to csv")
keys = full_list[0].keys()

with open(csv_file, "w", newline="") as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()

    dict_writer.writerows(full_list)

writing to csv
